$$ ITI \space AI-Pro: \space Intake \space 44 $$
$$ Recommender \space Systems $$
$$ Lab \space no. \space 3 $$

# `01` Import Necessary Libraries

## `i` Default Libraries

In [1]:
import numpy as np
import pandas as pd

## `ii` Additional Libraries
Add imports for additional libraries you used throughout the notebook

In [153]:
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.model_selection import train_test_split
import surprise

----------------------------

# `02` Load Data

Load `songsDataset.csv` file into a dataframe

In [38]:
data = pd.read_csv("Data/songsDataset.csv")
data.head()

,userID,songID,rating
0,0,90409,5
1,4,91266,1
2,5,8063,2
3,5,24427,4
4,5,105433,4


--------------------------

# `03` Matrix Factorization using Gradient Descent

Practice for Matrix Factorization Implementation

**Matrix Factorization Mathematical Derivation**

We know that the matrix factorization breaks the rating matrix $R$ into two matrices $U \in \mathbb{R}^{\#users \times K}$ and $M \in \mathbb{R}^{K \times \#items}$ where K represent the latent space dimensionality.

$R$ can then be approximated through the following equation:

$$
\mathbf{R} \approx \mathbf{U} \times \mathbf{M} = \hat{\mathbf{R}}
$$

The error, incorporating the regularization term, is calculated as follows:

$$
e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{u_{ik}m_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||U||^2 + ||M||^2)}
$$

In order to be able to use Stochastic Gradient Descent (SGD) to optimize $U$ and $M$, we need to find the partial derivatives of the error function with respect to both $u_{ik}$ and $m_{kj}$. The partial derivatives can be derived as follows:

$$
\frac{\partial}{\partial u_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(m_{kj}) + \frac{\beta}{2} \times (2 u_{ik}) = -2 e_{ij} m_{kj} + \beta u_{ik}
$$

$$
\frac{\partial}{\partial m_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(u_{ik}) + \frac{\beta}{2} \times (2 m_{kj}) = -2 e_{ij} u_{ik} + \beta m_{kj}
$$

Thus the update rules will be:

$$
u'_{ik} = u_{ik} + \alpha \frac{\partial}{\partial u_{ik}}e_{ij}^2 = u_{ik} - \alpha(-2 e_{ij} m_{kj} + \beta u_{ik} ) = u_{ik} + \alpha(2 e_{ij} m_{kj} - \beta u_{ik} )
$$

$$
m'_{kj} = m_{kj} + \alpha \frac{\partial}{\partial m_{kj}}e_{ij}^2 = m_{kj} - \alpha(-2 e_{ij} u_{ik} + \beta m_{kj} ) = m_{kj} + \alpha(2 e_{ij} u_{ik} - \beta m_{kj} )
$$

## `0` Construct Utility Matrix from the Data

In [114]:
utility_matrix=data.pivot_table(index='userID', columns='songID', values='rating', fill_value=0)
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
utility_matrix = None
utility_matrix

songID,2263,2726,3785,8063,12709,13859,16548,17029,19299,19670,...,113954,119103,120147,122065,123176,125557,126757,131048,132189,134732
userID,,,,,,,,,,,,,,,,,,,,,
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0
14,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
199976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,0.0,0.0
199980,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
199988,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,5.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## `i` Matrices Initialization

Initialize the two random weights matrices $U$ and $M$ (Try $K=3$)

**Note**: Refer to the next cell for the dimensions of $U$ and $M$

**Hine**: You may use a function from `numpy.random` module (see, [Documentation](https://numpy.org/doc/stable/reference/random/index.html))

In [40]:
users=len(utility_matrix)
items=len(utility_matrix.columns)
K=3

In [238]:
U = np.random.rand(users, K) * 0.1
M = np.random.rand(K, items) * 0.1

## `ii` Define a Function to Implement Matrix Factorization

**Function Parameters**:
- `R`: Utility Matrix [of shape: (no. of users, no. of items)]
- `U`: User Latent Features Array [of shape: (no. of users, K)]
- `M`: Items Latent Features Array [of shape: (K, no. of items)]
- `epochs`: No. of training epochs
- `lr`: Learning rate (alpha)
- `beta`: Regularization Parameter

**Function Output**:
- `U`: Optimized User Latent Features Array
- `M`: Optimized Items Latent Features Array

**Main Procedures**:
1. Calculate predicted ratings
2. Calculate MSE Error
3. Calculate gradients
4. Update $U$ and $M$


In [23]:
utility_matrix.shape[0]

53963

In [239]:
def matrix_factorization(R: np.ndarray, U: np.ndarray, M: np.ndarray, epochs: int, lr: float, beta: float):
    """
    Function Parameters:
    - `R`: Utility Matrix [of shape: `(no. of users, no. of items)`]
    - `U`: User Latent Features Array [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features Array [of shape: `(K, no. of items)`]
    - `epochs`: No. of training epochs
    - `lr`: Learning rate (alpha)
    - `beta`: Regularization Parameter

    Function Output:
    - `U`: Optimized User Latent Features Array
    - `M`: Optimized Items Latent Features Array
    """

    # Confirm that no. of features is consistent between U and M
    assert U.shape[1] == M.shape[0], f'U and M must have consistent K. Found K={U.shape[1]} for U and K={M.shape[0]} for M'

    # Extract No. of Features (K)
    K = U.shape[1]

    # Define the Epochs loop
    for epoch in range(epochs):
        # Loop over every element in R
        for i in range(R.shape[0]): # Loop over each user
            for j in range(R.shape[1]): # Loop over each item
                if R[i,j] !=0: # Only proceed if the current interaction (i, j) is not missing
                    eij = (R[i, j] - np.dot(U[i], M[:, j]))**2# Calculate the error in prediction
                    for k in range(K): # Loop over each latent features dimension
                        # Update Rules for both U and M:
                        eij+= beta/2* (U[i, k]**2 + M[k, j]**2)# calculate the second part of the error
                        grad_U = -2 * eij * M[k, j] + beta * U[i, k]
                        grad_M = -2 * eij * U[i, k] + beta * M[k, j]

                        # Apply gradient clipping to avoid large updates
                        U[i, k] = U[i, k] - lr * np.clip(grad_U, -0.5, 1)
                        M[k, j] = M[k, j] - lr * np.clip(grad_M, -0.5, 1)

                        # U[i, k] = U[i, k] - lr * grad_U
                        # M[k, j] = M[k, j] - lr * grad_M
                        
                        

        ## Error Calculation ##
        e_last = e if epoch > 0 else 100000000
        e = 0 # Initialize a variable to accumelate the errors
        #n = np.count_nonzero(R) # Count the number of non-zero elements in R
        for i in range(R.shape[0]): # Loop over each user
            for j in range(R.shape[1]): # Loop over each item
                if R[i,j]!=0: # Only proceed if the current interaction (i, j) is not missing
                                # since we only calculate the error for interactions having a ground truth value
                    first_part = (R[i, j] - np.dot(U[i], M[:, j]))**2# calculate the first part of the error
                    
                    second_part =0  # Initialize a variable to accumelate the second part of the error

                    for k in range(K): # Loop over each latent features dimension
                        second_part += beta/2* (U[i, k]**2 + M[k, j]**2)# calculate the second part of the error
                    e += (first_part + second_part )# accumelate the error to the total error
                    
        print(f'Epoch {epoch+1}/{epochs}: Total Error = {e}')

        if e < 0.001 or e_last-e < 0.001: # Stop if error is so small or improvement is not significant
            break

    return U, M


### `[Bonus]` Vectorized Error Calculation

Can the error calculation part be vectorized to get rid of for loops?

If you would like a challenge, try to redefine the function in the next cell with a vectorized error calculation.

## `iii` Use the Function to to Optimize the $U$ and $V$

In [240]:
U, M = matrix_factorization(R=utility_matrix.values, U=U, M=M, epochs=50, lr=0.01, beta=0.01)

Epoch 1/50: Total Error = 690870.8323882236
Epoch 2/50: Total Error = 474122.92681749153
Epoch 3/50: Total Error = 443858.6881059658
Epoch 4/50: Total Error = 780503.4896271516


## `iv` Recommend top-K Songs

Recommend top-K ($K=5$) songs for user ($userID=199988$)

Note: Make sure to filter songs they already rated before

In [241]:
#convert U to dataframe with userID as index 
U_df = pd.DataFrame(U, index=utility_matrix.index)
M_df = pd.DataFrame(M, columns=utility_matrix.columns)

In [242]:
def recommend_songs(user_id: int, U: pd.DataFrame, M: pd.DataFrame, utility_matrix: pd.DataFrame, top_n: int):
    """
    Function Parameters:
    - `user_id`: ID of the user for whom we want to recommend songs
    - `U`: User Latent Features DataFrame [of shape: `(no. of users, K)`]
    - `M`: Items Latent Features DataFrame [of shape: `(K, no. of items)`]
    - `utility_matrix`: Utility Matrix DataFrame [of shape: `(no. of users, no. of items)`]
    - `top_n`: No. of top songs to recommend

    Function Output:
    - `recommended_songs`: List of top `top_n` songs recommended for the user
    """

    # Extract the User Latent Features for the given user
    user_latent_features = U.loc[user_id]

    # Calculate the predicted ratings for the user
    predicted_ratings = pd.Series(np.dot(user_latent_features, M), index=M.columns)

    # Extract the songs that the user has already listened to
    songs_listened = utility_matrix.loc[user_id]
    songs_listened = songs_listened[songs_listened > 0].index

    # Remove the songs that the user has already listened to
    predicted_ratings = predicted_ratings.drop(songs_listened)

    # Sort the predicted ratings in descending order
    predicted_ratings = predicted_ratings.sort_values(ascending=False)

    # Extract the top N songs
    recommended_songs = predicted_ratings.head(top_n)

    # Print the top N recommended songs in the desired format
    print(f"Top {top_n} Recommended Items for User {user_id}:")
    for i, (song_id, pred_rating) in enumerate(recommended_songs.items(), start=1):
        print(f"\t- Top {i} Song: {song_id} (Predicted Rating: {pred_rating})")

    return recommended_songs

recommended_songs=recommend_songs(199988, U_df, M_df, utility_matrix, 5)

Top 5 Recommended Items for User 199988:
	- Top 1 Song: 22763 (Predicted Rating: 7.534969267705161)
	- Top 2 Song: 105433 (Predicted Rating: 7.398661760437866)
	- Top 3 Song: 123176 (Predicted Rating: 6.5302606462609685)
	- Top 4 Song: 13859 (Predicted Rating: 6.41121091294516)
	- Top 5 Song: 24427 (Predicted Rating: 6.297460740196029)


Top 5 Recommended Items for User 199988:
	- Top 1 Song: 122065 (Predicted Rating: 6.006507760135539)
	- Top 2 Song: 125557 (Predicted Rating: 5.819802641310125)
	- Top 3 Song: 52611 (Predicted Rating: 5.740148229067794)
	- Top 4 Song: 79622 (Predicted Rating: 5.701478248255747)
	- Top 5 Song: 71582 (Predicted Rating: 5.691077461653294)


------------------------

# `04` Matrix Factorization using SVD Algorithm

Practice for using `SVD` algorithm implementation from `scikit surprise` library.

## `i` Prepare the Data

- Load the Data into `surprise` Dataset
- Split data into train and test


In [151]:
#load the data into surprise dataset
df= pd.read_csv("Data/songsDataset.csv")
df.head()
reader = Reader(rating_scale=(0,5))
data =Dataset.load_from_df(df, reader)
trainset = data.build_full_trainset()
train, test = train_test_split(data, 0.2, random_state=42)

## `ii` Model Initialization

Instantiate two models:
- Model with baselines (biases)
- Model without baselines

**Note**: Use `surprise.prediction_algorithms.matrix_factorization.SVD` (see, [Documentation](https://surprise.readthedocs.io/en/stable/matrix_factorization.html#:~:text=surprise.prediction_algorithms.matrix_factorization.-,SVD,-(n_factors%3D)))

In [154]:
# Biased Model
biased_surprise_model=surprise.prediction_algorithms.matrix_factorization.SVD(n_factors=100, n_epochs=50, biased=True,init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02, lr_bu=None, lr_bi=None, lr_pu=None, lr_qi=None, reg_bu=None, reg_bi=None, reg_pu=None, reg_qi=None, random_state=None, verbose=False)

In [155]:
# Non-Biased Model
non_biased_surprise_model=surprise.prediction_algorithms.matrix_factorization.SVD(n_factors=100, n_epochs=50, biased=False,init_mean=0, init_std_dev=0.1, lr_all=0.005, reg_all=0.02, lr_bu=None, lr_bi=None, lr_pu=None, lr_qi=None, reg_bu=None, reg_bi=None, reg_pu=None, reg_qi=None, random_state=None, verbose=False)

## `iii` Fit each Model on Training Data

In [156]:
# Biased Model
biased_surprise_model.fit(train)

In [157]:
# Non-Biased Model
non_biased_surprise_model.fit(train)

## `iv` Test both Models on the Testing Data

Compare the errors of the two models using multiple error formulas.

**Note**: Refer to `surprise.accuracy` module (see, [Documentation](https://surprise.readthedocs.io/en/stable/accuracy.html))

In [161]:
# Biased Model
b_predictions=biased_surprise_model.test(test)
print(surprise.accuracy.fcp(b_predictions, verbose=True))
print(surprise.accuracy.mae(b_predictions, verbose=True))
print(surprise.accuracy.rmse(b_predictions, verbose=True))
print(surprise.accuracy.mse(b_predictions, verbose=True))


FCP:  0.5010
0.5010080414213385
MAE:  1.2844
1.2843538549307312
RMSE: 1.4821
1.4820794745705264
MSE: 2.1966
2.1965595689432473


In [162]:
# Non-Biased Model
nb_predictions=non_biased_surprise_model.test(test)
print(surprise.accuracy.fcp(nb_predictions, verbose=True))
print(surprise.accuracy.mae(nb_predictions, verbose=True))
print(surprise.accuracy.rmse(nb_predictions, verbose=True))
print(surprise.accuracy.mse(nb_predictions, verbose=True))

FCP:  0.4907
0.4906527013339824
MAE:  1.9095
1.9095005082716978
RMSE: 2.2983
2.298251386148651
MSE: 5.2820
5.281959433934196


## `v` Recommend Top $10$ Songs for User $199988$

Is there a difference in recommended songs from the two models?

In [163]:
# Biased Model
## `v` Recommend Top $10$ Songs for User $199988$
v=199988
v_predictions=biased_surprise_model.test(trainset.build_anti_testset())
v_predictions=[(i.uid, i.iid, i.est) for i in v_predictions]
v_predictions=pd.DataFrame(v_predictions, columns=['userID', 'songID', 'rating'])
v_predictions=v_predictions[v_predictions['userID']==v]
v_predictions=v_predictions.sort_values(by='rating', ascending=False)
v_predictions.head(10)

,userID,songID,rating
2949739,199988,71582,4.913568
2949745,199988,126757,4.451734
2949720,199988,90409,4.443502
2949760,199988,42781,4.404766
2949752,199988,13859,4.377523
2949761,199988,125557,4.296523
2949766,199988,94535,4.283416
2949722,199988,8063,4.236408
2949746,199988,55240,4.182241
2949769,199988,3785,4.177182


In [164]:
# Non-Biased Model
v=199988
v_predictions=non_biased_surprise_model.test(trainset.build_anti_testset())
v_predictions=[(i.uid, i.iid, i.est) for i in v_predictions]
v_predictions=pd.DataFrame(v_predictions, columns=['userID', 'songID', 'rating'])
v_predictions=v_predictions[v_predictions['userID']==v]
v_predictions=v_predictions.sort_values(by='rating', ascending=False)
v_predictions.head(10)

,userID,songID,rating
2949720,199988,90409,3.752272
2949726,199988,105421,3.212084
2949752,199988,13859,3.014862
2949764,199988,25182,2.899753
2949751,199988,40712,2.711894
2949742,199988,22763,2.644944
2949724,199988,105433,2.640351
2949753,199988,72309,2.508197
2949735,199988,119103,2.432860
2949768,199988,123176,2.430241


In [165]:
#biased model and non biased model produce different song recommendations

----------------------------------------------

$$ Wish \space you \space all \space the \space best \space ♡ $$
$$ Abdelrahman \space Eid $$